# Analyzing Pulse-Chase Designs

This notebook will show the most suitable commands to work with pulse-chase data.

Throughout this notebook, we will be using the GRAND-SLAM processed SLAM-seq data set from Herzog et al. 2017 [[1]](https://www.nature.com/articles/nmeth.4435). The data set includes SLAM-seq pulse-chase samples of mouse embryonic stem cells (mESCs) with pulse durations of 0, 0.5, 1, 3, 6, 12 and 24 hours.

First we load the data and do the standard preprocessing steps. For more on these initial commands see the [Loading data and working with GrandPy objects](notebook_03_loading_data_and_working_with_GrandPy_objects.ipynb) notebook.

In this step, we need to modify the sample names in the data, as the names were not processed using systematic naming conventions. If systematic sample names were used, the `rename_sample` parameter would not be necessary. This parameter allows us to specify a function for renaming the samples. The specified function removes all occurrences of ".chase" and replaces 0.5 with 0_5. For example, "mESC.0.5h.chase.B" becomes "mESC.0_5h.B" and "mESC.0h.chase.C" becomes "mESC.0h.C". This enables us to load the data using the 'design'.

In [ ]:
import re
import pandas as pd
import grandpy as gp

dense = gp.read_grand("https://zenodo.org/record/7612564/files/chase_notrescued.tsv.gz", design=("Condition", "dur.4sU", "Replicate"),                     rename_sample=lambda v: re.sub(r"\.chase", "", re.sub(r"0\.5h", "0_5h", re.sub(r"nos4U","no4sU",v))))

dense = dense.filter_genes()
dense = dense.normalize()

The `fit_kinetics` method can be used to analyze pulse-chase experiment data and estimate the synthesis and degradation rates of genes. To perform this analysis, the 'fit_type' parameter of the function should be set to "chase".

In addition to the required input, `fit_kinetcis` also has several optional parameters that allow for customization of the analysis. For further information on these parameters, please refer to the [kinetic modeling](notebook_02_kinetic_modeling(progressive_labeling_time_courses).ipynb) notebook.

`fit_kinetics` uses the information in the GrandPy object and the specified parameters to fit a non-linear least squares model that describe the pulse-chase data. After fitting the models, a GrandPy object is returned with added analysis, that contains the inferred synthesis and degradation rates.

In [ ]:
dense = dense.fit_kinetics(fit_type="chase", max_processes = 6)

To speed up kinetic fitting, the `fit_kinetics` function uses multiple processes by default. For more information see the `fit_kinetics` docstring or refer to the [kinetic modeling](notebook_02_kinetic_modeling(progressive_labeling_time_courses).ipynb) notebook.

The function `get_analysis_table` can be used to retrieve the results from this analysis.

In [ ]:
dense.get_analysis_table().head()

We can also visualize single genes graphically (this is the same gene as used in Fig 4a in Herzog et al. 2017 [[1]](https://www.nature.com/articles/nmeth.4435):

In [ ]:
from grandpy import plot_gene_progressive_timecourse, plot_scatter

plot_gene_progressive_timecourse(dense, gene=(dense.get_genes("Dnmt3b", regex=True)[0]), fit_type="chase")

The curves represent the fitted model for this gene. Note that we do not specify the gene name directly, since here the gene names were labeled differently. The `get_genes` method can be used to find genes using a regex:

In [ ]:
dense.get_genes("Dnmt3b", regex=True)

In this step, we compare the half-life values obtained from `fit_kinetics` with those from Herzog et al. The latter was estimated by observing the decrease in mismatches during the chase period, while our method also considers the GRAND-SLAM-estimated NTR and posterior. To compare, we add Herzog et al's half-live values to our analyses and create a scatter plot with half-life values on both the x and y axes. The plot clearly shows a correlation between the two sets of results, indicating that the `fit_kinetics` results are consistent with those from Herzog et al.

In [ ]:
# WORK IN PROGRESS - ?

t = pd.read_csv("https://zenodo.org/record/7612564/files/halflifes.tsv?download=1", sep="\t", header=0).assign(Gene=lambda df: df["Name"].astype(str) + "." + df["End"].astype(str))
t.set_index("Gene", inplace=True)
t.index.name = "Symbol"

dense = dense.with_analysis("kinetics", table=t)

print(dense.get_analyses(description=True))

In [ ]:
plot_scatter(dense, 'Half-life (h)', 'mESC_Half-life')